<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/10_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#

In [ ]:
# 初始化記錄觸發情況的列表
record = []

# 根據 col1 和 col2 進行分組
for i, data in df.groupby(['col1', 'col2']):
    # 執行每個規則
    trigger1 = rule1(data)
    trigger2 = rule2(data)
    trigger3 = rule3(data)
    trigger4 = rule4(data)
    trigger5 = rule5(data)
    trigger6 = rule6(data)

    # 建立每個群組的規則觸發記錄
    temp = {
        'col1': i[0],  # col1 的值
        'col2': i[1],  # col2 的值
        'Rule1': 1 if len(trigger1) > 0 else 0,
        'Rule2': 1 if len(trigger2) > 0 else 0,
        'Rule3': 1 if len(trigger3) > 0 else 0,
        'Rule4': 1 if len(trigger4) > 0 else 0,
        'Rule5': 1 if len(trigger5) > 0 else 0,
        'Rule6': 1 if len(trigger6) > 0 else 0,
    }

    # 將結果加入記錄列表
    record.append(temp)

# 將記錄轉換為 DataFrame
record_df = pd.DataFrame(record)

# 顯示結果
print(record_df)

In [ ]:
# 移除 'col2' 列，只保留規則的觸發情況
record_df_no_col2 = record_df.drop(columns=['col2'])

# 根據 col1 分組，計算每個規則的觸發率 (平均值)
rule_rate_df = record_df_no_col2.groupby('col1').mean()

# 顯示規則觸發率
print(rule_rate_df)

# 版2

In [ ]:

# 初始化結果列表
result = []

# 定義要執行的規則
rules = {
    'Rule1': rule1,
    'Rule2': rule2,
    'Rule3': rule3,
    'Rule4': rule4,
    'Rule5': rule5,
    'Rule6': rule6
}

# 根據 col1 和 col2 進行分群
grouped = df.groupby(['col1', 'col2'])

# 遍歷每一個群組
for group_key, group_data in grouped:
    group_result = {'col1': group_key[0], 'col2': group_key[1]}

    # 遍歷每個規則，並檢查是否觸發
    for rule_name, rule_func in rules.items():
        result_data = rule_func(group_data)  # 執行規則
        group_result[rule_name] = 1 if len(result_data) > 0 else 0  # 觸發為1，否則為0

    # 將每個群組的結果加入列表
    result.append(group_result)

# 將結果轉為 DataFrame
result_df = pd.DataFrame(result)

# 顯示結果
print(result_df)


# function : 組合各種rule

In [ ]:
import pandas as pd

# 定義函數
def calculate_trigger_rate(df, col_list):
    """
    根據輸入的規則欄位列表，計算每個 col1 下規則的觸發率。

    Parameters:
    df (pd.DataFrame): 包含規則觸發信息的 DataFrame。
    col_list (list): 包含規則名稱的列表，例如 ['Rule1', 'Rule2', 'Rule5']。

    Returns:
    pd.DataFrame: 返回每個 col1 在規則列表下的觸發率。
    """
    # 先計算 col_list 中第一個規則的觸發率
    initial_rule = col_list[0]
    initial_rate = df.groupby('col1')[initial_rule].mean().rename(f'{initial_rule}_rate')

    # 計算指定規則列表的觸發情況 (邏輯“或”運算)
    df['combined_rule'] = df[col_list].any(axis=1).astype(int)

    # 計算 col1 在所有規則條件下的觸發率
    combined_rate = df.groupby('col1')['combined_rule'].mean().rename(f'{"_".join(col_list)}_rate')

    # 將兩個觸發率合併在一起返回
    result_df = pd.concat([initial_rate, combined_rate], axis=1)
    return result_df

# 測試資料
record_df = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'C', 'C'],
    'Rule1': [1, 1, 0, 0, 1, 1, 0],
    'Rule2': [1, 0, 0, 1, 1, 0, 1],
    'Rule3': [0, 0, 1, 0, 1, 0, 1],
    'Rule4': [1, 1, 1, 0, 1, 1, 0],
    'Rule5': [0, 0, 1, 1, 0, 0, 1],
    'Rule6': [1, 0, 1, 0, 1, 0, 0]
}

df = pd.DataFrame(record_df)

# 呼叫函數並指定規則列表 ['Rule1', 'Rule2', 'Rule5']
result = calculate_trigger_rate(df, ['Rule1', 'Rule2', 'Rule5'])

# 顯示結果
print(result)
